In [23]:
import pandas as pd
import xarray as xr
import numpy as np
import netCDF4 as nc
#from scipy.spatial import distance

In [24]:
# Open catalogue
df = pd.read_csv('/home/data/ReAnalysis/ERA5/Storm_analysis/NAECv1/NAEC_1979_2020_v1.csv')

# open netcdf mask file
file = '/pampa/picart/Masks/mask_GEM5_ERA5grid'
data = xr.open_dataset(file)

# export netcdf to dataframe and drop index
mk = data.to_dataframe().reset_index()

# Only keep grid points coordinates that are within CRCM6 domain
mk = mk.loc[mk.HU == True]

# open crcm6 boundary layer 
bnd = pd.read_csv('/pampa/cloutier/outline_crcm6_domain.csv', index_col = 0)

In [34]:
# Chen filtre les ETC qui sont pendant au moins 24h CONSÉCUTIVES dans le domaine et dont le centre est
# à une distance minimale de 5° du bord du domaine de CRCM6.

# on doit donc déterminer une fonction qui calcule la distance entre un point de grille et le bord du 
# domaine de CRCM6

# Il faut que le point de grille ait une valeur HU == True et qu'il soit è une distance minimale de 5° de 
# tous les points de grille qui tracent la limite 
mk = mk.rename(columns={'lat' : 'latitude', 'lon' : 'longitude'})
merge = df.merge(mk, how='left', on=['latitude', 'longitude'])
merge = merge.fillna(value = False)
df24_consec = pd.DataFrame(columns = df.columns)

In [37]:
# Fonction qui détermine si le point donné dans le catalogue est à une distance 
# minimale de tous les points de grille du frame du domaine de crcm6 de 5°

def get_distance(latS, lonS, bnd) :
    dist_cond = True

    for _, row2 in bnd.iterrows():
        latD = row2['lat']
        lonD = row2['lon']
        dist = ((latS-latD)**2 + (lonS - lonD)**2)**0.5
        
        if dist < 5 : 
            dist_min = False
            break
            
    return dist_cond

In [85]:
# Iterate through each storm
for storm_id in merge['storm'].unique():
    storm_data = merge[merge['storm'] == storm_id].copy()
    count = 0
   
    # Iterate through each grid point
    for _, row in storm_data.iterrows() :
        dist_cond = False
        hu = row['HU']
        latS = row['latitude']
        lonS = row['longitude']
        
        # If the grid point is in the subdomain 
        if hu : 
            dist_cond = get_distance(latS, lonS, bnd)
    
        # No need to put and HU == True in the next if condition, because if dist_cond = true, 
        # it means that the if hu conditions was respected (dist_cond is reset to false for each 
        # grid point iteration )
        if dist_cond : 
            count += 1

        if hu == False and count >= 24 :
            break

        if hu == False and count < 24 :
            count = 0

    if count >= 24 : 
        df24_consec = df24_consec.append(storm_data)
        print('Year in process : ', df24_consec['datetime'].iloc[-1])
        print('Storm : ', df24_consec['storm'].iloc[-1])
        

Year in process :  1979010204
Storm :  1
Year in process :  1979010218
Storm :  3
Year in process :  1979010822
Storm :  12
Year in process :  1979011015
Storm :  13


KeyboardInterrupt: 

In [31]:

        mk_filtered = mk.loc[(mk['latitude'] <= latS - 5) | (mk['latitude'] >= latS + 5) |
                             (mk['longitude'] <= lonS - 5) | (mk['longitude'] >= lonS + 5)]

28

In [91]:
import pandas as pd
import math
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np
import xarray as xr

""" 

    Maxine Cloutier-Gervais

Created : 

    June 7th, 2023

Info : 
    
    This code creates and filters, a file that contains ETC that were active for 24 consecutives 
    hours or more in the crcm6 domain.

"""

def open_cat_mask(cat_in, bnd_in, mask_in) : 

    """
    Open NAEC catalogue, boundary file  and transform netCDF mask into dataframe 

    Parameters  : 
        cat_in  : path of the catalogue csv file
        mask_in : path of the mask netCDF file
        bnd_in  : path of the csv file that defines the boundary grid points
                  of CRCM6 domain
                    
    Returns : 
        cat : Dataframe containing NAEC catalogue data
        mk  : Dataframe containing mask data
        bnd : Dataframe containing boundary grid points 

    """
    
    # Step 1 : Open catalogue and boundary csv file
    print('lecture cat...')
    cat = pd.read_csv(cat_in)
    print('lecture bnd...')
    bnd = pd.read_csv(bnd_in, index_col = 0)

    # Step 2 : Open mask netCDF file and convert into dataframe
    print('lecture mask...')
    mk = xr.open_dataset(mask_in)
    mk = mk.to_dataframe()

    # Step 3 :  Drop index lat lon, but keep columns
    mk = mk.reset_index()

    # Step 4 : Rename lat & lon columns for latitude & longitude
    mk = mk.rename(columns={'lat' : 'latitude', 'lon' : 'longitude'})

    return cat, bnd, mk


def get_distance(latS, lonS, bnd) : 

    """
    Determine if a given grid point is at a minimal distance of 5deg from
    all CRCM6 boundary grid point domain

    Paramters : 
        latS  : Latitude of the catalogue grid point
        lonS  : Longitude of the catalogue grid point
        bnd   : Dataframe containing boundary grid points

    Returns       : 
        dist_cond : True if all grid points are within a minimal distance of 5deg
                    from all boundary layer grid points and False if not.
    """

    dist_cond = True
    
    # filter out boundary grid points to restrict search
    bnd_filt = bnd.loc[(bnd['lat'] < latS - 6) | (bnd['lat'] > latS + 6) |
                       (bnd['lon'] < lonS - 6) | (bnd['lon'] > lonS + 6)]
    
    for _, row1 in bnd_filt.iterrows():
        latD = row1['lat']
        lonD = row1['lon']
        dist = ((latS-latD)**2 + (lonS - lonD)**2)**0.5
        
        if dist < 5 : 
            dist_min = False
            break
            
    return dist_cond




def add_season(df, output_file) : 

    """
    Add a column called 'season' in df24 that gives the season in which the ETC occured. 
    If the ETC occured in two or more season, the chosen season will be the one in which 
    the ETC has the most grid point

    DJF : December, January & November
    MAM : March, April & May
    JJA : June, July & April
    SON : September, October and December
    
    Parameters : 
        df (dataframe) : Dataframe to which we want to add the season column

    returns : 
        df_new : 
    """

    seasons = { 'SON': [9, 10, 11], 'DJF': [12, 1, 2], 'MAM': [3, 4, 5], 'JJA': [6, 7, 8] }

    # Step 1 : Add 'month' column in dataframe 

    df['month'] = (df.datetime // 10000) % 100

    # Step 2 : Group the storms by their ID and count the number of grid point 
    #          in each month

    storm_seasons = df.groupby(['storm', 'month']).size().unstack().fillna(0)

    # Step 3 : Determine the month with the maximum grid points for each storm

    storm_seasons['season'] = storm_seasons.idxmax(axis=1)
    
    # Step 4 : Transform month number into season
    
    storm_seasons['season'] = storm_seasons['season'].map(
    lambda month: next((season for season, months in seasons.items() if month in months), None)
    )
    
    # Step 5 : Merge the season column into original dataframe
    
    df_new = df.merge(storm_seasons['season'], on='storm', how='left')

    # Step 6 : Delete month column

    df_new = df_new.drop(['month'], axis = 1)
    
    # Step 7 : move season column next to datetime (TODO)
    
    #df_new.insert(3, 'season', df_new.pop('season'))

    return df_new



""" MAIN PROGRAM """


# Step 1 : Open catalogue, boundary catalogue and mask
#cat_in = ('/home/data/ReAnalysis/ERA5/Storm_analysis/NAECv1/NAEC_1979_2020_v1.csv')
#bnd_in = ('/pampa/cloutier/outline_crcm6_domain.csv')
#mask_in = ('/pampa/picart/Masks/mask_GEM5_ERA5grid')

#cat, bnd, mk = open_cat_mask(cat_in, bnd_in, mask_in)

# Step 2 : Merge cat and mask to add HU column in cat

merge = cat.merge(mk, how='left', on=['latitude', 'longitude'])
merge = merge.fillna(value = False)

# Step 3 : Initialize empty dataframe that will contain the final result

df24 = pd.DataFrame(columns = cat.columns)

# Step 4 : Filter catalogue data

# Iterate through each storm 
for storm_id in merge['storm'].unique():
    storm_data = merge[merge['storm'] == storm_id].copy() # copy of merge for the given storm
    count = 0 # lifetime count
    stInDom=[]

    # Iterate through each grid point of the storm
    for _, row in storm_data.iterrows() : 
        print('first for loop : ', row['storm'])
        dist_cond = False 
        hu = row['HU']
        latS = row['latitude']
        lonS= row['longitude']
        
        # check if storm center is within subdomain and at a 5° minimal 
        # distance from boundaries
        print('check distance ...')
        cond = get_distance(latS, lonS, bnd) 
        stInDom.append(cond)
        print('finish first loop, stInDom = ', stInDom)
    
    # add a new column that determines if each storm center agrees or not with the above condition
    print('adding StInDom in storm_data ...')
    storm_data['StInDom'] = stInDom
    print(storm_data['StInDom'])
    n = 23
    # exclude the last 23 lines in the search
    storm_data_rows = storm_data.head(len(storm_data) - n)
    
    count = 0
    
    for idx, row in storm_data_rows.iterrows() : 
        print('second for loop : ', row['storm'])
        if row ['StInDom'] : 
            print('Initializing count to one')
            count = 1
            
            # keep iterating from the StInDom == True found
            for _, row in islice(storm_data.iterrows(), idx, None) : 
                if row['StInDom'] : 
                    print('storm is in domain')
                    count += 1
                
                else :  
                    break
            
            if count >= 24 : 
                break
    print(count)                           
    if count >= 24 :
        df24 = df24.append(storm_data)
        print('Year in process : ', df24['datetime'].iloc[-1])              

first for loop :  1
check distance ...
finish first loop, stInDom =  [True]
first for loop :  1
check distance ...
finish first loop, stInDom =  [True, True]
first for loop :  1
check distance ...
finish first loop, stInDom =  [True, True, True]
first for loop :  1
check distance ...
finish first loop, stInDom =  [True, True, True, True]
first for loop :  1
check distance ...
finish first loop, stInDom =  [True, True, True, True, True]
first for loop :  1
check distance ...
finish first loop, stInDom =  [True, True, True, True, True, True]
first for loop :  1
check distance ...
finish first loop, stInDom =  [True, True, True, True, True, True, True]
first for loop :  1
check distance ...
finish first loop, stInDom =  [True, True, True, True, True, True, True, True]
first for loop :  1
check distance ...
finish first loop, stInDom =  [True, True, True, True, True, True, True, True, True]
first for loop :  1
check distance ...
finish first loop, stInDom =  [True, True, True, True, True, 

finish first loop, stInDom =  [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
first for loop :  2
check distance ...
finish first loop, stInDom =  [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
first for loop :  2
check distance ...
finish first loop, stInDom =  [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
first for loop :  2
check distance ...
finish first loop, stInDom =  [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
first for loop :  2
check distance ...
finish first loop, stInDom =  [True, True, True, True, True,

AttributeError: 'DataFrame' object has no attribute 'StInDom'

SyntaxError: invalid syntax (<ipython-input-73-1c38c93b6a0e>, line 1)

     storm  lifetime    datetime  latitude  longitude  MSLPmin   VORSmax  \
742     15         1  1979010902     47.75     298.75   1002.0  0.000098   
743     15         2  1979010903     47.75     299.00   1002.0  0.000098   
744     15         3  1979010904     48.00     299.50   1002.0  0.000096   
745     15         4  1979010905     47.50     298.75   1003.0  0.000085   
746     15         5  1979010906     46.75     297.50   1004.0  0.000067   
747     15         6  1979010907     47.25     298.25   1003.0  0.000068   
748     15         7  1979010908     46.25     297.00   1003.0  0.000059   
749     15         8  1979010909     46.50     297.25   1003.0  0.000061   
750     15         9  1979010910     47.00     298.00   1003.0  0.000066   
751     15        10  1979010911     47.25     298.75   1003.0  0.000067   
752     15        11  1979010912     47.50     299.00   1002.0  0.000067   
753     15        12  1979010913     47.75     299.50   1003.0  0.000066   
754     15  